In [17]:
import pandas as pd
import requests

#    "UserID": "290EFCBB-FAEF-4B15-AED6-BBFB7B290205",


BEA_API_KEY = "YOUR_BEA_KEY"  # replace with your key
URL = "https://apps.bea.gov/api/data/"

params = {
    "UserID": "290EFCBB-FAEF-4B15-AED6-BBFB7B290205",
    "method": "GetData",          # REQUIRED
    "datasetname": "Regional",    # BEA Regional accounts
    "TableName": "CAGDP5",        # GDP by County & Industry (NAICS, chained $2017)
#    "LineCode": "1",              # 1 = Total GDP (you can remove to get all lines)
    "Year": "2022",               # pick a year
    "GeoFIPS": "COUNTY",          # ALL U.S. counties; use 'CA' for California only
    "ResultFormat": "JSON"
}

r = requests.get(URL, params=params, timeout=60)
r.raise_for_status()

payload = r.json()

In [18]:
# Defensive parsing: show the full error if Results isn't present
if not isinstance(payload, dict) or "BEAAPI" not in payload:
    raise RuntimeError(f"Unexpected response:\n{payload}")

beaapi = payload["BEAAPI"]
if "Results" not in beaapi:
    # Print helpful error details
    raise RuntimeError(f"BEA API error:\n{beaapi.get('Error', beaapi)}")

In [19]:
data = beaapi["Results"]["Data"]
df = pd.DataFrame(data)

KeyError: 'Data'

In [20]:
# Optional: keep only county rows (5-digit FIPS) and tidy up
df = df[df["GeoFips"].str.len() == 5].copy()

In [21]:
# DataValue is a string like '1,234' → make numeric (millions of chained 2017 $)
df["DataValue_num"] = pd.to_numeric(df["DataValue"].str.replace(",", ""), errors="coerce")

In [11]:
print(df.head(), df.shape)

       Code GeoFips      GeoName TimePeriod               CL_UNIT UNIT_MULT  \
0  CAGDP2-1   01001  Autauga, AL       2022  Thousands of dollars         3   
1  CAGDP2-1   01003  Baldwin, AL       2022  Thousands of dollars         3   
2  CAGDP2-1   01005  Barbour, AL       2022  Thousands of dollars         3   
3  CAGDP2-1   01007     Bibb, AL       2022  Thousands of dollars         3   
4  CAGDP2-1   01009   Blount, AL       2022  Thousands of dollars         3   

  DataValue NoteRef  DataValue_num  
0   2318914     NaN        2318914  
1  11097699     NaN       11097699  
2    931174     NaN         931174  
3    571231     NaN         571231  
4   1266269     NaN        1266269   (3118, 9)


In [22]:
df

,Code,GeoFips,GeoName,TimePeriod,CL_UNIT,UNIT_MULT,DataValue,NoteRef,DataValue_num
0,CAGDP2-1,01001,"Autauga, AL",2022,Thousands of dollars,3,2318914,NaN,2318914
1,CAGDP2-1,01003,"Baldwin, AL",2022,Thousands of dollars,3,11097699,NaN,11097699
2,CAGDP2-1,01005,"Barbour, AL",2022,Thousands of dollars,3,931174,NaN,931174
3,CAGDP2-1,01007,"Bibb, AL",2022,Thousands of dollars,3,571231,NaN,571231
4,CAGDP2-1,01009,"Blount, AL",2022,Thousands of dollars,3,1266269,NaN,1266269
...,...,...,...,...,...,...,...,...,...
3113,CAGDP2-1,56037,"Sweetwater, WY",2022,Thousands of dollars,3,4311769,NaN,4311769
3114,CAGDP2-1,56039,"Teton, WY",2022,Thousands of dollars,3,4225374,NaN,4225374
3115,CAGDP2-1,56041,"Uinta, WY",2022,Thousands of dollars,3,1062478,NaN,1062478
3116,CAGDP2-1,56043,"Washakie, WY",2022,Thousands of dollars,3,415865,NaN,415865


In [ ]:
df.to_csv("/Users/michaelwalker/RDM_Datalab/bea_output_by_county_